In [1]:
import numpy as np
import cv2 as cv
import random
from tqdm import tqdm

In [2]:
# Global variables
MIN_CHOICE = 0
MAX_CHOICE = 8

# population size and number of generations
population_size = 10000
generation_num = 50
parent_num = 30

# A list of lambda functions to optimize direction finding
ST = lambda x,y: (x,y)
UL = lambda x,y: (x-1, y-1)
UU = lambda x,y: (x-1, y)
UR = lambda x,y: (x-1, y+1)
RR = lambda x,y: (x, y+1)
DR = lambda x,y: (x+1, y+1)
DD = lambda x,y: (x+1, y)
DL = lambda x,y: (x+1, y-1)
LL = lambda x,y: (x, y-1)

DIRECTION_FINDERS = [ST, UL, UU, UR, RR, DR, DD, DL, LL]

In [3]:
def read_binary_img(filename = 'assets/ex1.png'):
    img = cv.imread(filename, cv.IMREAD_GRAYSCALE)
    _, im_bw = cv.threshold(img, 128, 255, cv.THRESH_BINARY)
    return np.invert(im_bw.astype(dtype=bool))

In [4]:
def similarity_score(src, tgt, c1 = 0.75, c2 = 0.25):
    """
    calculates a score for similarity between the given arrays
    `src` and `tgt`

    :param src: the first array to compare
    :param tgt: the second array to compare
    :param c1: a coefficient for matching cells
    :param c2: a coefficient for non-matching cells
    """
    
    # count the ones and zeros in the target image
    ones = np.count_nonzero(tgt)
    zeros = tgt.size - ones
    
    # count the ones and zeros matching the target
    matching_ones = np.count_nonzero((src == True) & (src == tgt))
    matching_zeros = np.count_nonzero((src == False) & (src == tgt))

    return c1 * matching_ones / ones + c2 * matching_zeros / zeros

In [5]:
def calc_path_len(tgt):
    ones = np.count_nonzero(tgt)
    diag = max(tgt.shape)
    
    # the legth of the path is the black pixels + the max
    # steps needed to reach start position
    return {
        'max': min(ones * 2 + diag, tgt.size),
        'min': ones
    }

In [6]:
def execute_path(path, img_shape = (20, 20)):
    """
    returns a numpy array with `path` executed on it
    turning each element passed through to True
    """
    blank = np.zeros(img_shape, dtype=bool)
    x, y = blank.shape[0]- 1, 0
    blank[x][y] = True
    for s in path:
        x_new, y_new = DIRECTION_FINDERS[s](x, y)
        try:
            blank[x_new][y_new] = True
            x, y = x_new, y_new
        except IndexError:
            continue
    return blank

In [7]:
def generate_random_path(size, min_steps):
    path_start = np.random.randint(MIN_CHOICE + 1, MAX_CHOICE, size = min_steps)
    path_rest = np.random.randint(MIN_CHOICE, MAX_CHOICE, size = size - min_steps)
    return np.concatenate((path_start, path_rest))

In [8]:
def fitness(path, tgt):
    result_img = execute_path(path, tgt.shape)
    return similarity_score(result_img, tgt)

In [9]:
def crossover(parents, offspring_size):
    # crossover index
    c_idx = np.uint8(offspring_size/2)
    
    # the best parents mating is not always the best option
    np.random.shuffle(parents)
    num_parents = len(parents)
    
    offspring = [np.concatenate((parents[k % num_parents][0: c_idx], parents[(k+1) % num_parents][c_idx:])) for k in range(offspring_size)]
    
    return offspring

In [10]:
def mutate(population, mu = 0.01):
    path_len = population[0].shape[0]
    
    for indiv_idx in range(len(population)):
        indices = random.sample(range(path_len), int(path_len * mu))
        for step_idx in indices:
            r = (population[indiv_idx][step_idx] + np.random.randint(0, 8)) % (MAX_CHOICE + 1)
            population[indiv_idx][step_idx] = r
    return population

In [11]:
def plot_path(path, img_shape=(20,20)):
    img = np.invert(execute_path(path, img_shape))
    img = img.astype(np.uint8)  #convert to an unsigned byte
    img *= 255
    cv.imshow('Indices',img)
    cv.waitKey(0)
    cv.destroyAllWindows() 

In [12]:
img = read_binary_img()
path = calc_path_len(img)

In [13]:
paths = [generate_random_path(path['max'], path['min']) for _ in range(population_size)]

In [14]:
paths.sort(reverse = True, key = lambda p: fitness(p, img))

In [15]:
fitness(paths[0], img)

0.5478667459301262

In [16]:
%%time
pbar = tqdm(total=generation_num)
for _ in range(generation_num):
    parents = paths[:parent_num]
    offspring = crossover(parents, offspring_size = population_size - parent_num)
    offspring = mutate(offspring)
    paths = parents + offspring
    paths.sort(reverse = True, key = lambda p: fitness(p, img))
    pbar.update(1)
pbar.close()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [03:41<00:00,  4.44s/it]

CPU times: user 3min 41s, sys: 146 ms, total: 3min 42s
Wall time: 3min 41s


In [17]:
fitness(paths[0], img)

0.763387141027986

In [ ]:
plot_path(paths[0], img_shape = img.shape)